<a href="https://colab.research.google.com/github/otanet/Quantum_Computing_Annealing_Machine_20211209/blob/main/tsp_20211209.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Travelling Salesman Problem

Amplify を用いた巡回セールスマン問題の解法について解説します。

## 巡回セールスマン問題の定式化

[巡回セールスマン問題](https://en.wikipedia.org/wiki/Travelling_salesman_problem) とは、いくつかの都市からなる集合とそれぞれの都市間の距離が与えられたときに、
全ての都市を通ってはじめの都市に戻ってくる巡回路のうち、移動距離の合計が最小となるものを求める組合せ最適化問題です。


![240px-GLPK_solution_of_a_travelling_salesman_problem.svg.png](https://upload.wikimedia.org/wikipedia/commons/thumb/1/11/GLPK_solution_of_a_travelling_salesman_problem.svg/240px-GLPK_solution_of_a_travelling_salesman_problem.svg.png)

イジングマシンを用いるためには通る都市の順番を二値多変数多項式で表す必要があります。

そのために、都市数を $N$ として、$N\times N$ の変数テーブルを用意します。変数テーブルの各行は都市を何番目に通るかに対応し、各列は各都市に対応します。
テーブルの各要素にバイナリ変数を割り当て、$n$行$i$列にある変数が$1$となっているとき、セールスマンは$n$番目に$i$番目の都市を通ると解釈します。

例えば、4都市の場合、経路 $A\rightarrow C\rightarrow B\rightarrow D \rightarrow A$ を表現するには、次のような $4\times 4$ の表を用意します。

| turn| A | B | C | D |
|-----|---|---|---|---|
| 1st | 1 | 0 | 0 | 0 |
| 2nd | 0 | 0 | 1 | 0 |
| 3rd | 0 | 1 | 0 | 0 |
| 4th | 0 | 0 | 0 | 1 |


上記の表における各変数を経路の順番 $n$ と都市のインデックス $i$ を用いて$q_{n,i}$ と表すことにし、
都市 $i$ と都市 $j$ の間の移動距離を $d_{ij}$ と表します。

このとき、総移動距離は

$$
\sum_{n=0}^{N-1}{\sum_{i=0}^{N-1}{\sum_{j=0}^{N-1}{ d_{ij} q_{n, i} q_{n+1, j} }}}
$$

で表されます。

後でプログラムコード化する都合により、インデックスは $0$ からスタートしていることに注意してください。
また、$n=N-1$ の場合に $n+1=N$ となってインデックスがはみ出てしまいますが、$q_N=q_0$ だと思うことにより、最後の都市から最初の都市への移動距離だと解釈することができます。
実装するときは、インデックスを $N$ で割った余りを使用することにします。

しかしこれだけでは定式化として不十分です。なぜなら、上記の変数テーブルは「全ての都市を通る」という制約と「同時に一つだけ通る」という制約が考慮されていないためです。
極端な例として、最初の都市から動かないという組合せも許されています。そこで変数テーブルの全ての行と列に対して次のような制約を課します。

$$
    \sum_{i=0}^{N-1}{q_{n, i}} = 1 \quad n \in \left\{0, 1, \cdots, N - 1 \right\}
$$
$$
    \sum_{n=0}^{N-1}{q_{n, i}} = 1 \quad i \in \left\{0, 1, \cdots, N - 1 \right\}
$$

これは変数テーブルの取り得る値のうち、各行と各列に $1$ がちょうど $1$ 回しか現れない制約を意味します。
各行に $1$ がちょうど $1$ 回現れる制約はセールスマンが $1$ 度に訪れる都市は $1$ つであるという条件に対応し、
各列に  $1$ がちょうど $1$ 回現れる制約は各都市をちょうど $1$ 回ずつ訪れるという条件に対応しています。

上記をまとめると、次の二値多変数多項式の最小値を求めれば良いことがわかりました。

- コスト関数
$$
\sum_{n=0}^{N-1}{\sum_{i=0}^{N-1}{\sum_{j=0}^{N-1}{ d_{ij} q_{n, i} q_{n+1, j} }}}
$$

- 制約条件
$$
    \sum_{i=0}^{N-1}{q_{n, i}} = 1 \quad n \in \left\{0, 1, \cdots, N - 1 \right\}
$$
$$
    \sum_{n=0}^{N-1}{q_{n, i}} = 1 \quad i \in \left\{0, 1, \cdots, N - 1 \right\}
$$


## 問題の生成

まず巡回セールスマン問題の入力となる、各都市間の距離を作成します。ここでは `numpy` を用いて、二次元平面上のランダムな位置に都市を配置して距離行列を生成します。次のように、都市数を与えることでランダムな巡回セールスマン問題を生成する関数を作成しました。

In [ ]:
! pip install -q numpy

In [ ]:
import numpy as np


def gen_random_tsp(ncity: int):
    # 座標
    locations = np.random.uniform(size=(ncity, 2))

    # 距離行列
    all_diffs = np.expand_dims(locations, axis=1) - np.expand_dims(locations, axis=0)
    distances = np.sqrt(np.sum(all_diffs ** 2, axis=-1))

    return locations, distances

`gen_random_tsp` 関数の二つ目の返り値である`distances` は上記定式化における $d$ に相当します。

次のようにして $32$ 都市について各都市の座標がプロットされます。

In [ ]:
! pip install -q matplotlib

In [ ]:
import matplotlib.pyplot as plt


def show_plot(locs: np.ndarray):
    plt.figure(figsize=(7, 7))
    plt.xlabel("x")
    plt.ylabel("y")
    plt.scatter(*zip(*locations))
    plt.show()


ncity = 32
locations, distances = gen_random_tsp(ncity)

In [ ]:
show_plot(locations)

## 二値多項式模型の構築

次に変数テーブルを作成します。これが巡回路における訪問順と訪問先を表します。

In [ ]:
! pip install -q amplify

In [ ]:
from amplify import BinarySymbolGenerator

gen = BinarySymbolGenerator()
q = gen.array(ncity, ncity)
q

作成した変数テーブルを用いてコスト関数 $\sum_{n=0}^{N-1}{\sum_{i=0}^{N-1}{\sum_{j=0}^{N-1}{ d_{ij} q_{n, i} q_{n+1, j} }}}$ を構築します。

In [ ]:
cost = 0
for n in range(ncity):
    for i in range(ncity):
        for j in range(ncity):
            cost += distances[i, j] * q[n, i] * q[(n + 1) % ncity, j]

次のようにすれば `numpy` と同様の記法で高速にコスト関数が構築できます。

In [ ]:
from amplify import sum_poly, einsum, newaxis

# 配列演算を用いる方法
f = sum_poly(ncity, lambda n: (distances * q[n, :, newaxis] * q[n - 1]).sum())

# einsum 関数を用いる方法
f = einsum("ij,ni,nj->", distances, q, q.roll(-1, axis=0))

次に制約条件を構築します。One-hot 制約は `one_hot()` 関数で作成されます。
各行各列 $N$ 個ずつの制約条件をリストとして作成し、
`sum()` 関数で全ての制約条件を足し合わせます。

In [ ]:
from amplify.constraint import one_hot

# 行に対する制約
row_constraints = [one_hot(q[n]) for n in range(ncity)]

# 列に対する制約
col_constraints = [one_hot(q[:, i]) for i in range(ncity)]

constraints = sum(row_constraints) + sum(col_constraints)

最後にコスト関数と全ての制約条件を足し合わせて論理模型オブジェクトを作成します。ここで制約条件の強さに注意する必要があります。
適切な制約条件の強さはコスト関数に依存し、十分に大きな値にする必要があるからです。
ここでは、十分な大きな値として距離行列の最大値を設定します。詳細はリファレンス [1] を参照してください。

In [ ]:
constraints *= np.amax(distances)  # 制約条件の強さを設定
model = cost + constraints


[1]: [K. Takehara, D. Oku, Y. Matsuda, S. Tanaka and N. Togawa, "A Multiple Coefficients Trial Method to Solve Combinatorial Optimization Problems for Simulated-annealing-based Ising Machines," 2019 IEEE 9th International Conference on Consumer Electronics (ICCE-Berlin), Berlin, Germany, 2019, pp. 64-69, doi: 10.1109/ICCE-Berlin47944.2019.8966167.](https://ieeexplore.ieee.org/abstract/document/8966167)


## イジングマシンの実行

イジングマシンのクライアントを作成しパラメータを設定します。その後ソルバーを作成しクライアントを設定します。

In [ ]:
from amplify import Solver
from amplify.client import FixstarsClient

client = FixstarsClient()
client.parameters.timeout = 5000  # タイムアウト5秒
client.token = "トークンを入力してください"

solver = Solver(client)

次のようにしてイジングマシンを実行し結果を取得します。

In [ ]:
result = solver.solve(model)
if len(result.solutions) == 0:
    raise RuntimeError("Any one of constraints is not satisfied.")

energy = result.solutions[0].energy
values = result.solutions[0].values

### Note

もし `result.solutions` オブジェクトが空のリストの場合、制約条件を満たす解が得られなかったことを意味します。
この場合はイジングマシンのパラメータや制約の強さの変更が必要です。

## 結果の解析

`energy` はコスト関数の評価値を表します。今回の定式化では移動距離の合計に相当します。

In [ ]:
energy

`values` は入力変数と解の値のマッピングを表す辞書です。そのままでは評価しづらいので、次のようにして変数テーブル `q` と同一の形式にデコードします。

In [ ]:
q_values = q.decode(values)
# q_values

```python
>>> q_values # 4都市の場合の実行例
[[1, 0, 0, 0],
 [0, 0, 0, 1],
 [0, 0, 1, 0],
 [0, 1, 0, 0]]
```

これを見ると、各行・各列に $1$ が一回ずつしか現れていないため、確かに制約条件を満たしていることがわかります。 各行について、$1$ が現れる列インデックスを取得すれば経路がわかるので、次のように `numpy` 関数を用いて調べます

In [ ]:
route = np.where(q_values == 1)[1]
print(route)

最後に、得られた経路を表示します。次の関数でプロットできます。

In [ ]:
def show_route(route: list, distances: np.ndarray, locations: np.ndarray):

    ncity = len(route)
    path_length = sum(
        [distances[route[i]][route[(i + 1) % ncity]] for i in range(ncity)]
    )

    x = [i[0] for i in locations]
    y = [i[1] for i in locations]
    plt.figure(figsize=(7, 7))
    plt.title(f"path length: {path_length}")
    plt.xlabel("x")
    plt.ylabel("y")

    for i in range(ncity):
        r = route[i]
        n = route[(i + 1) % ncity]
        plt.plot([x[r], x[n]], [y[r], y[n]], "b-")
    plt.plot(x, y, "ro")
    plt.show()

    return path_length

In [ ]:
show_route(route, distances, locations)

## プログラミング課題

### 課題1
上で求めた解は、始点 (出発する都市) を定めていないために回転対称性を持ちます。
例えば、 $A\rightarrow B \rightarrow C \rightarrow D \rightarrow A$ という経路が最良解であるならば、
$B\rightarrow C \rightarrow D \rightarrow A \rightarrow B$ という経路も最良解となり、解空間が冗長性を持ってしまっています。
始点を固定することにより、問題を縮約化し必要となる変数の数を $(N-1)^2$ に減らすことができます。

最初に出発する都市を0番目の都市に固定して巡回セールスマン問題を解いてください。
また、このとき論理模型において使用されている変数の数が $N^2$ よりも少なくなっていることを確認してください。

ヒント：
  * 例えば `q[0, 0]=0` とすると、バイナリ変数 $q_{0,0} = 0$ に固定することができます。目的関数や制約条件を作成する前に設定してください。
  * 論理模型において使用されている変数の数は `model.num_logical_vars` で取得できます。

### 課題2
最長単純道問題とは、いくつかの都市が与えられたとき、ある都市から出発してすべての都市をちょうど1回ずつ通って別の都市まで行く経路のうち最も長いものを求める問題です。
巡回セールスマン問題を参考にして、最長単純道問題を解いてください。